# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [1]:
import pandas as pd
import numpy as np

---
## 2. Load data

In [2]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [3]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [4]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [5]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as yo can!

In [6]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()


In [7]:
monthly_price_by_id 

,id,price_date,price_off_peak_var,price_off_peak_fix
0,0002203ffbb812588b632b9e628cc38d,2015-01-01,0.126098,40.565969
1,0002203ffbb812588b632b9e628cc38d,2015-02-01,0.126098,40.565969
2,0002203ffbb812588b632b9e628cc38d,2015-03-01,0.128067,40.728885
3,0002203ffbb812588b632b9e628cc38d,2015-04-01,0.128067,40.728885
4,0002203ffbb812588b632b9e628cc38d,2015-05-01,0.128067,40.728885
...,...,...,...,...
192997,ffff7fa066f1fb305ae285bb03bf325a,2015-08-01,0.119916,40.728885
192998,ffff7fa066f1fb305ae285bb03bf325a,2015-09-01,0.119916,40.728885
192999,ffff7fa066f1fb305ae285bb03bf325a,2015-10-01,0.119916,40.728885
193000,ffff7fa066f1fb305ae285bb03bf325a,2015-11-01,0.119916,40.728885


In [8]:
jan_prices = monthly_price_by_id[monthly_price_by_id['price_date'].dt.month==1]

In [9]:
jan_prices

,id,price_date,price_off_peak_var,price_off_peak_fix
0,0002203ffbb812588b632b9e628cc38d,2015-01-01,0.126098,40.565969
12,0004351ebdd665e6ee664792efc4fd13,2015-01-01,0.148047,44.266931
24,0010bcc39e42b3c2131ed2ce55246e3c,2015-01-01,0.150837,44.444710
36,0010ee3855fdea87602a5b7aba8e42de,2015-01-01,0.123086,40.565969
48,00114d74e963e47177db89bc70108537,2015-01-01,0.149434,44.266931
...,...,...,...,...
192942,ffef185810e44254c3a4c6395e6b4d8a,2015-01-01,0.162720,41.063970
192954,fffac626da707b1b5ab11e8431a4d0a2,2015-01-01,0.148825,44.266931
192966,fffc0cacd305dd51f316424bbb08d1bd,2015-01-01,0.153159,41.063970
192978,fffe4f5646aa39c7f97f95ae2679ce64,2015-01-01,0.127566,40.565969


In [10]:
dec_prices = monthly_price_by_id[monthly_price_by_id['price_date'].dt.month==12]

In [11]:
dec_prices 

,id,price_date,price_off_peak_var,price_off_peak_fix
11,0002203ffbb812588b632b9e628cc38d,2015-12-01,0.119906,40.728885
23,0004351ebdd665e6ee664792efc4fd13,2015-12-01,0.143943,44.444710
35,0010bcc39e42b3c2131ed2ce55246e3c,2015-12-01,0.201280,45.944710
47,0010ee3855fdea87602a5b7aba8e42de,2015-12-01,0.113068,40.728885
59,00114d74e963e47177db89bc70108537,2015-12-01,0.145440,44.266930
...,...,...,...,...
192953,ffef185810e44254c3a4c6395e6b4d8a,2015-12-01,0.112488,40.728885
192965,fffac626da707b1b5ab11e8431a4d0a2,2015-12-01,0.145047,44.444710
192977,fffc0cacd305dd51f316424bbb08d1bd,2015-12-01,0.151399,41.228885
192989,fffe4f5646aa39c7f97f95ae2679ce64,2015-12-01,0.118175,40.728885


In [12]:
dec_prices1 = monthly_price_by_id.groupby('id').last().reset_index()

In [13]:
dec_prices1

,id,price_date,price_off_peak_var,price_off_peak_fix
0,0002203ffbb812588b632b9e628cc38d,2015-12-01,0.119906,40.728885
1,0004351ebdd665e6ee664792efc4fd13,2015-12-01,0.143943,44.444710
2,0010bcc39e42b3c2131ed2ce55246e3c,2015-12-01,0.201280,45.944710
3,0010ee3855fdea87602a5b7aba8e42de,2015-12-01,0.113068,40.728885
4,00114d74e963e47177db89bc70108537,2015-12-01,0.145440,44.266930
...,...,...,...,...
16091,ffef185810e44254c3a4c6395e6b4d8a,2015-12-01,0.112488,40.728885
16092,fffac626da707b1b5ab11e8431a4d0a2,2015-12-01,0.145047,44.444710
16093,fffc0cacd305dd51f316424bbb08d1bd,2015-12-01,0.151399,41.228885
16094,fffe4f5646aa39c7f97f95ae2679ce64,2015-12-01,0.118175,40.728885


In [14]:
# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'price_off_peak_var_dec', 'price_off_peak_fix': 'price_off_peak_fix_dec'}), jan_prices.drop(columns='price_date'), on='id')

In [15]:
diff['offpeak_diff_dec_january_energy'] = diff['price_off_peak_var_dec'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['price_off_peak_fix_dec'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


### Combining the clean data and group data

In [16]:
df = pd.merge(df, diff, on='id')
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,9.084737e-04,2.086294,99.530517,44.235794,2.086425,9.953056e+01,4.423670e+01,1,0.020057,3.700961
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000000e+00,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000e+00,0,-0.003767,0.177779
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000000e+00,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000e+00,0,-0.004670,0.177779
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.000000e+00,0.000000,0.000000,0.000000,0.000003,0.000000e+00,0.000000e+00,0,-0.004547,0.177779
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,4.860000e-10,0.000000,0.000000,0.000000,0.000011,2.896760e-06,4.860000e-10,0,-0.006192,0.162916


In [17]:
# Monthly consumption difference
df['cons_diff'] = df['cons_12m'] - df['cons_last_month']

In [18]:
df['cons_diff']

0            0
1         4660
2          544
3         1584
4         3899
         ...  
14579    32270
14580     7042
14581     1665
14582      131
14583     8730
Name: cons_diff, Length: 14584, dtype: int64

In [19]:

df['tenure'] = ((df['date_end'] - df['date_activ'])/ np.timedelta64(1, 'Y')).astype(int)


In [20]:
df['tenure']

0        3
1        7
2        6
3        6
4        6
        ..
14579    3
14580    4
14581    3
14582    4
14583    6
Name: tenure, Length: 14584, dtype: int32

In [21]:
df.drop(columns=['date_end', 'date_activ'], inplace=True)

In [22]:
df = df.drop(columns=['cons_12m', 'cons_last_month'])


In [23]:
df.columns

Index(['id', 'channel_sales', 'cons_gas_12m', 'date_modif_prod',
       'date_renewal', 'forecast_cons_12m', 'forecast_cons_year',
       'forecast_discount_energy', 'forecast_meter_rent_12m',
       'forecast_price_energy_off_peak', 'forecast_price_energy_peak',
       'forecast_price_pow_off_peak', 'has_gas', 'imp_cons',
       'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max',
       'var_year_price_off_peak_var', 'var_year_price_peak_var',
       'var_year_price_mid_peak_var', 'var_year_price_off_peak_fix',
       'var_year_price_peak_fix', 'var_year_price_mid_peak_fix',
       'var_year_price_off_peak', 'var_year_price_peak',
       'var_year_price_mid_peak', 'var_6m_price_off_peak_var',
       'var_6m_price_peak_var', 'var_6m_price_mid_peak_var',
       'var_6m_price_off_peak_fix', 'var_6m_price_peak_fix',
       'var_6m_price_mid_peak_fix', 'var_6m_price_off_peak',
       'var_6m_price_peak', 'var_6m_price

In [24]:
df.head()

,id,channel_sales,cons_gas_12m,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,...,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,cons_diff,tenure
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,54946,2015-11-01,2015-06-23,0.00,0,0.0,1.78,0.114481,...,99.530517,44.235794,2.086425,9.953056e+01,4.423670e+01,1,0.020057,3.700961,0,3
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,0,2009-08-21,2015-08-31,189.95,0,0.0,16.27,0.145711,...,0.000000,0.000000,0.009485,1.217891e-03,0.000000e+00,0,-0.003767,0.177779,4660,7
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,0,2010-04-16,2015-04-17,47.96,0,0.0,38.72,0.165794,...,0.000000,0.000000,0.000004,9.450150e-08,0.000000e+00,0,-0.004670,0.177779,544,6
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,0,2010-03-30,2015-03-31,240.04,0,0.0,19.83,0.146694,...,0.000000,0.000000,0.000003,0.000000e+00,0.000000e+00,0,-0.004547,0.177779,1584,6
4,149d57cf92fc41cf94415803a877cb4b,MISSING,0,2010-01-13,2015-03-09,445.75,526,0.0,131.73,0.116900,...,0.000000,0.000000,0.000011,2.896760e-06,4.860000e-10,0,-0.006192,0.162916,3899,6


In [25]:
# Days between renewal and modification
df['days_to_renewal'] = (df['date_renewal'] - df['date_modif_prod']).dt.days

In [26]:
# Extract year, month, day from both dates (assuming they are datetime)
df['modif_prod_year'] = df['date_modif_prod'].dt.year
df['modif_prod_month'] = df['date_modif_prod'].dt.month
df['modif_prod_day'] = df['date_modif_prod'].dt.day

df['renewal_year'] = df['date_renewal'].dt.year
df['renewal_month'] = df['date_renewal'].dt.month
df['renewal_day'] = df['date_renewal'].dt.day

In [27]:
df.drop(columns=['date_modif_prod', 'date_renewal'], inplace=True)

In [28]:
df.head()

,id,channel_sales,cons_gas_12m,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,...,offpeak_diff_dec_january_power,cons_diff,tenure,days_to_renewal,modif_prod_year,modif_prod_month,modif_prod_day,renewal_year,renewal_month,renewal_day
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,54946,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,...,3.700961,0,3,-131,2015,11,1,2015,6,23
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,0,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,...,0.177779,4660,7,2201,2009,8,21,2015,8,31
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,0,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,...,0.177779,544,6,1827,2010,4,16,2015,4,17
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,0,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,...,0.177779,1584,6,1827,2010,3,30,2015,3,31
4,149d57cf92fc41cf94415803a877cb4b,MISSING,0,445.75,526,0.0,131.73,0.116900,0.100015,40.606701,...,0.162916,3899,6,1881,2010,1,13,2015,3,9


In [29]:
# Separate features (X) and target variable (y) - 
X = df.drop('churn', axis=1)  
y = df['churn']

In [30]:
from scipy.stats import pearsonr

# Create a correlation matrix
corr_matrix = X.corr()

# Set a threshold for correlation (e.g., 0.5)
correlation_threshold = 0.5

# Filter features with correlation above the threshold (absolute value)
correlated_features = [col for col in corr_matrix.columns if 
                       any(corr_matrix[col].abs() > correlation_threshold) & (col != 'churn')]

# Keep only the correlated features
X_filtered_corr = X[correlated_features]


C:\Users\pawan.thakre\AppData\Local\Temp\ipykernel_17284\1799260530.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = X.corr()


In [31]:
df_selected=X_filtered_corr

In [32]:
df_selected

,cons_gas_12m,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,imp_cons,margin_gross_pow_ele,...,offpeak_diff_dec_january_power,cons_diff,tenure,days_to_renewal,modif_prod_year,modif_prod_month,modif_prod_day,renewal_year,renewal_month,renewal_day
0,54946,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,0.00,25.44,...,3.700961,0,3,-131,2015,11,1,2015,6,23
1,0,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,0.00,16.38,...,0.177779,4660,7,2201,2009,8,21,2015,8,31
2,0,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,0.00,28.60,...,0.177779,544,6,1827,2010,4,16,2015,4,17
3,0,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,0.00,30.22,...,0.177779,1584,6,1827,2010,3,30,2015,3,31
4,0,445.75,526,0.0,131.73,0.116900,0.100015,40.606701,52.32,44.91,...,0.162916,3899,6,1881,2010,1,13,2015,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14579,47940,4648.01,0,0.0,18.57,0.138305,0.000000,44.311378,0.00,27.88,...,0.177779,32270,3,-347,2015,5,8,2014,5,26
14580,0,631.69,181,0.0,144.03,0.100167,0.091892,58.995952,15.94,0.00,...,0.236694,7042,4,1096,2012,8,27,2015,8,28
14581,0,190.39,179,0.0,129.60,0.116900,0.100015,40.606701,18.05,39.84,...,0.162916,1665,3,1097,2012,2,8,2015,2,9
14582,0,19.34,0,0.0,7.18,0.145711,0.000000,44.311378,0.00,13.08,...,0.177779,131,4,1096,2012,8,30,2015,8,31


In [33]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14584 entries, 0 to 14583
Data columns (total 44 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   cons_gas_12m                     14584 non-null  int64  
 1   forecast_cons_12m                14584 non-null  float64
 2   forecast_cons_year               14584 non-null  int64  
 3   forecast_discount_energy         14584 non-null  float64
 4   forecast_meter_rent_12m          14584 non-null  float64
 5   forecast_price_energy_off_peak   14584 non-null  float64
 6   forecast_price_energy_peak       14584 non-null  float64
 7   forecast_price_pow_off_peak      14584 non-null  float64
 8   imp_cons                         14584 non-null  float64
 9   margin_gross_pow_ele             14584 non-null  float64
 10  margin_net_pow_ele               14584 non-null  float64
 11  nb_prod_act                      14584 non-null  int64  
 12  net_margin        

### Feature Selection 

In [34]:
count=0
for i in df_selected.columns:
    count=count+1
    print(f" {count}  the unique value of columns {i} : {df_selected[i].unique()}")
    print()

 1  the unique value of columns cons_gas_12m : [  54946       0 1542867 ...    2126    3036   47940]

 2  the unique value of columns forecast_cons_12m : [  0.   189.95  47.96 ... 190.39  19.34 762.41]

 3  the unique value of columns forecast_cons_year : [    0   526  1998 ...  2187 12321  3269]

 4  the unique value of columns forecast_discount_energy : [ 0. 30. 24. 28. 26. 22. 10. 25. 19. 17. 23.]

 5  the unique value of columns forecast_meter_rent_12m : [  1.78  16.27  38.72 ... 165.13   7.59   7.18]

 6  the unique value of columns forecast_price_energy_off_peak : [0.114481  0.145711  0.165794  0.146694  0.1169    0.164775  0.166178
 0.115174  0.115761  0.164637  0.167086  0.115237  0.162327  0.168242
 0.144902  0.164058  0.11286   0.118636  0.142417  0.160302  0.143478
 0.144039  0.14317   0.198096  0.143166  0.116509  0.114604  0.11536
 0.120372  0.147506  0.142881  0.145877  0.11691   0.114025  0.0006
 0.115744  0.144149  0.186188  0.145884  0.113323  0.16304   0.112852
 0.145

In [35]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [36]:
# Apply MinMaxScaler
scaler = MinMaxScaler()
df_scaled_minmax = df_selected.copy()
df_scaled_minmax = scaler.fit_transform(df_selected)

In [37]:
df_scaled_minmax

array([[1.32253724e-02, 0.00000000e+00, 0.00000000e+00, ...,
        6.66666667e-01, 4.54545455e-01, 7.33333333e-01],
       [0.00000000e+00, 2.29123662e-03, 0.00000000e+00, ...,
        6.66666667e-01, 6.36363636e-01, 1.00000000e+00],
       [0.00000000e+00, 5.78508599e-04, 0.00000000e+00, ...,
        6.66666667e-01, 2.72727273e-01, 5.33333333e-01],
       ...,
       [0.00000000e+00, 2.29654404e-03, 1.02066999e-03, ...,
        6.66666667e-01, 9.09090909e-02, 2.66666667e-01],
       [0.00000000e+00, 2.33285160e-04, 0.00000000e+00, ...,
        6.66666667e-01, 6.36363636e-01, 1.00000000e+00],
       [0.00000000e+00, 9.19642912e-03, 0.00000000e+00, ...,
        6.66666667e-01, 1.00000000e+00, 6.66666667e-01]])

In [38]:
df_selected

,cons_gas_12m,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,imp_cons,margin_gross_pow_ele,...,offpeak_diff_dec_january_power,cons_diff,tenure,days_to_renewal,modif_prod_year,modif_prod_month,modif_prod_day,renewal_year,renewal_month,renewal_day
0,54946,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,0.00,25.44,...,3.700961,0,3,-131,2015,11,1,2015,6,23
1,0,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,0.00,16.38,...,0.177779,4660,7,2201,2009,8,21,2015,8,31
2,0,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,0.00,28.60,...,0.177779,544,6,1827,2010,4,16,2015,4,17
3,0,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,0.00,30.22,...,0.177779,1584,6,1827,2010,3,30,2015,3,31
4,0,445.75,526,0.0,131.73,0.116900,0.100015,40.606701,52.32,44.91,...,0.162916,3899,6,1881,2010,1,13,2015,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14579,47940,4648.01,0,0.0,18.57,0.138305,0.000000,44.311378,0.00,27.88,...,0.177779,32270,3,-347,2015,5,8,2014,5,26
14580,0,631.69,181,0.0,144.03,0.100167,0.091892,58.995952,15.94,0.00,...,0.236694,7042,4,1096,2012,8,27,2015,8,28
14581,0,190.39,179,0.0,129.60,0.116900,0.100015,40.606701,18.05,39.84,...,0.162916,1665,3,1097,2012,2,8,2015,2,9
14582,0,19.34,0,0.0,7.18,0.145711,0.000000,44.311378,0.00,13.08,...,0.177779,131,4,1096,2012,8,30,2015,8,31


## Model Building

In [39]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [40]:
# Separate target variable from independent variables
y
X = df_selected
print(X.shape)
print(y.shape)

(14584, 44)
(14584,)


In [41]:
# Train Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [42]:
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)


(10938, 44) (3646, 44)
(10938,) (3646,)


In [43]:
model = RandomForestClassifier(
    n_estimators=1000
)
model.fit(X_train, y_train)


RandomForestClassifier(n_estimators=1000)

#param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],   # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],   # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]      # Minimum number of samples required to be at a leaf node
}

In [ ]:
#from sklearn.model_selection import train_test_split, GridSearchCV
#from sklearn.metrics import accuracy_score
#grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
#grid_search.fit(X_train, y_train)

In [44]:
predictions = model.predict(X_test)
tn, fp, fn, tp = metrics.confusion_matrix(y_test, predictions).ravel()

In [45]:
y_test.value_counts()

0    3300
1     346
Name: churn, dtype: int64

In [46]:
print(f"True positives: {tp}")
print(f"False positives: {fp}")
print(f"True negatives: {tn}")
print(f"False negatives: {fn}\n")

print(f"Accuracy: {metrics.accuracy_score(y_test, predictions)}")
print(f"Precision: {metrics.precision_score(y_test, predictions)}")
print(f"Recall: {metrics.recall_score(y_test, predictions)}")

True positives: 24
False positives: 3
True negatives: 3297
False negatives: 322

Accuracy: 0.9108612177729019
Precision: 0.8888888888888888
Recall: 0.06936416184971098
